Vector Search for Fast Deduplication and Batch Annotation of Text Classification Records

In [1]:
from datasets import load_dataset

agnews = load_dataset("ag_news")

agnews_train, agnews_valid = (
    agnews["train"].train_test_split(test_size=7600, seed=43).values()
)


Using custom data configuration default
Reusing dataset ag_news (/home/ufuk/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached split indices for dataset at /home/ufuk/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-0fb8fdf64cc3fcc4.arrow and /home/ufuk/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-80423eae5320d3cb.arrow


In [2]:
%pip install faiss-cpu sentence_transformers transformers datasets

Note: you may need to restart the kernel to use updated packages.


In [4]:
import argilla as rg

In [5]:
from sentence_transformers import SentenceTransformer

In [6]:
# instantiate the model for the sentence embeddings
# we strongly recommend using a GPU for the computation of the embeddings
model = SentenceTransformer("all-mpnet-base-v2", device="cuda")

In [7]:
text = "Abraham Lincoln stressed the value of data centric AI and adviced to use Argilla"

In [8]:
embedding = model.encode(text)

In [11]:
embedding

array([-5.53162768e-03,  9.18077826e-02, -2.50837747e-02, -2.14579497e-02,
        5.91655215e-03,  7.32013956e-03,  5.14037348e-02, -2.65303981e-02,
       -9.91398282e-03,  9.39362682e-03,  2.26464495e-02,  2.10230555e-02,
       -2.52178684e-02,  2.95277741e-02, -3.35275149e-03,  1.37330573e-02,
        2.12650262e-02,  2.43432671e-02,  2.94778333e-03,  5.02051087e-03,
       -3.18038352e-02, -3.60914059e-02, -1.45315938e-02, -1.51758771e-02,
       -4.77715768e-02, -2.57307831e-02, -2.46327631e-02, -8.47828691e-04,
        1.89807545e-02, -6.85988832e-03, -1.16309384e-02, -5.40443659e-02,
        1.64653454e-02,  4.63529956e-03,  1.64105984e-06,  1.25938039e-02,
        2.26511061e-03,  7.39694573e-03, -7.80298859e-02,  3.62519734e-02,
        4.77010533e-02,  1.18941262e-01,  1.93802472e-02,  1.16939368e-02,
       -7.17506483e-02,  7.58235008e-02,  2.52142064e-02, -1.65963061e-02,
       -1.00047113e-02,  4.64385338e-02,  1.22358231e-03,  1.97744071e-02,
        9.23999175e-02, -

In [12]:
embedding.tolist()

[-0.005531627684831619,
 0.0918077826499939,
 -0.02508377470076084,
 -0.02145794965326786,
 0.005916552152484655,
 0.007320139557123184,
 0.05140373483300209,
 -0.026530398055911064,
 -0.009913982823491096,
 0.00939362682402134,
 0.022646449506282806,
 0.021023055538535118,
 -0.025217868387699127,
 0.029527774080634117,
 -0.003352751489728689,
 0.01373305730521679,
 0.021265026181936264,
 0.02434326708316803,
 0.0029477833304554224,
 0.005020510870963335,
 -0.031803835183382034,
 -0.036091405898332596,
 -0.014531593769788742,
 -0.015175877138972282,
 -0.047771576792001724,
 -0.025730783119797707,
 -0.024632763117551804,
 -0.0008478286908939481,
 0.018980754539370537,
 -0.006859888322651386,
 -0.01163093838840723,
 -0.054044365882873535,
 0.01646534539759159,
 0.004635299555957317,
 1.641059839130321e-06,
 0.01259380392730236,
 0.0022651106119155884,
 0.007396945729851723,
 -0.07802988588809967,
 0.0362519733607769,
 0.04770105332136154,
 0.11894126236438751,
 0.019380247220396996,
 0.0

In [ ]:
# build our labelled records to evaluate our heuristic rules and optimize the thresholds
records = [
    rg.TextClassificationRecord(
        text=record["text"],
        metadata={"split": "labelled"},
        embeddings={
            "text_embedding": {
                "vector": model.encode(record["text"]).tolist()
            }
        },
        annotation=agnews_valid.features["label"].int2str(record["label"]),
        id=f"valid_{idx}",
    )
    for idx, record in enumerate(agnews_valid)
]

# build our unlabelled records
records += [
    rg.TextClassificationRecord(
        text=record["text"],
        embeddings={
            "text_embedding": {
                "vector": model.encode(record["text"]).tolist()
            }
        },
        metadata={"split": "unlabelled"},
        id=f"train_{idx}",
    )
    for idx, record in enumerate(agnews_train)
]


In [ ]:
# log the records to Argilla
rg.log(records, name="news2")